In [1]:
import torch
import torch.nn as nn
from transformers import pipeline

classifier = pipeline("text-classification")

text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely,
Bumblebee."""

output = classifier(text)
output

/Users/nurma/vscode_projects/NLP/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'NEGATIVE', 'score': 0.9015459418296814}]

In [2]:
import pandas as pd

ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


,entity_group,score,word,start,end
0,ORG,0.879009,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556568,Mega,208,212
4,PER,0.590257,##tron,212,216
5,ORG,0.669693,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775360,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


In [3]:
pd.DataFrame(outputs).sort_values("score")

,entity_group,score,word,start,end
6,MISC,0.498349,##icons,259,264
3,MISC,0.556568,Mega,208,212
4,PER,0.590257,##tron,212,216
5,ORG,0.669693,Decept,253,259
7,MISC,0.775360,Megatron,350,358
9,PER,0.812096,Bumblebee,502,511
0,ORG,0.879009,Amazon,5,11
8,MISC,0.987854,Optimus Prime,367,380
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97


In [4]:
reader = pipeline("question-answering")
question = "what is the name of the customer?"

output = reader(question=question, context=text)
output

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


{'score': 0.19141174852848053,
 'start': 491,
 'end': 511,
 'answer': 'Sincerely,\nBumblebee'}

In [5]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
Your min_length=56 must be inferior than your max_length=45.
/Users/nurma/vscode_projects/NLP/.venv/lib/python3.13/site-packages/transformers/generation/utils.py:1633: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead.


In [8]:
model_checkpoint = "glazzova/ml_translation_model1"
translator = pipeline("translation", model=model_checkpoint)

outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

ImportError: 
MarianTokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [7]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely,
Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up.

This order was placed on my order list and I have received the following email message from your email address.

I believe it is reasonable to assume your order was shipped by parcel. I am going to be contacting you for your refund of your order.

Thank you.

I would like to provide you with one final word, that I am happy to have received your emails about the package. I hope that you will

In [ ]:
from transformers import pipeline

# model = pipeline("text-classification", model="distilbert-base-uncased-finetuned-emotion") from local files
model_id = "NurekeKZ/distilbert-base-uncased-finetuned-emotion"
model = pipeline("text-classification", model=model_id)

Device set to use cuda:0


In [8]:
text_test = "fuck you"
output = model(text_test, return_all_scores=True)
output

c:\Users\sshuser\NLP_for_transformers\.venv\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 0.07460010796785355},
  {'label': 'LABEL_1', 'score': 0.03884405270218849},
  {'label': 'LABEL_2', 'score': 0.017287995666265488},
  {'label': 'LABEL_3', 'score': 0.8360769748687744},
  {'label': 'LABEL_4', 'score': 0.02508675679564476},
  {'label': 'LABEL_5', 'score': 0.008104161359369755}]]

In [7]:
model.push_to_hub(repo_id="NurekeKZ" , commit_message="Inital commit")

model.safetensors: 100%|██████████| 268M/268M [00:14<00:00, 18.0MB/s]   


CommitInfo(commit_url='https://huggingface.co/NurekeKZ/NurekeKZ/commit/f19118691867c10b58b21d5ae688909d92623fe0', commit_message='Inital commit', commit_description='', oid='f19118691867c10b58b21d5ae688909d92623fe0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/NurekeKZ/NurekeKZ', endpoint='https://huggingface.co', repo_type='model', repo_id='NurekeKZ/NurekeKZ'), pr_revision=None, pr_num=None)

In [ ]:
model_id = "NurekeKZ/distilbert-base-uncased-finetuned-emotion"